# UNET - Train on Fashion Dataset


This notebook shows how to train UNET on the Fashion dataset. 

The code of the *Fashion* dataset is included below. A maskrcnn.utils.DataSet object is customized for the Fashion dataset.  This object is used to train the model.

## Set Backend For GPU Support
This can be commented out if an Nvidia GPU is being used

In [ ]:
import os
# fix libiomp5.dylib error for mac
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# OPTIONAL SETTING
# Here we override the keras backend env variable to use plaidml
# plaidml can make use of AMD GPUs 
# This assignment needs to be added before loading keras libraries

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# to install plaidML, activate appropriate environment and then:
#   pip install -U plaidml-keras
#   plaidml-setup

import matplotlib.pyplot as plt
import numpy as np
import keras
keras.__version__

### Load Modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon


# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from mrcnn.visualize import display_images

%matplotlib inline 

In [ ]:
# !pip install ipywidgets 
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

# %%capture
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
import skimage
import random
# from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from pycocotools.coco import maskUtils

### Setup local variables

In [ ]:
data_dir    = '../../data/FashionDataSet/'
train_dir   = os.path.join(data_dir, 'train')
images_file = os.path.join(data_dir, 'train.csv' )
cat_file    = os.path.join(data_dir, 'label_descriptions.json' )

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(data_dir, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(data_dir, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

### Defin UNET model Structure

In [1]:
# U-Net architecture
def UNet(input_img):

    # Contracting Path
    enc1 = Conv2D(64, (3,3), activation='relu', padding='same')(input_img)
    enc1 = Conv2D(64, (3,3), activation='relu', padding='same')(enc1)
    
    enc2 = MaxPooling2D(pool_size=(2,2))(enc1)
    enc2 = Conv2D(128, (3,3), activation='relu', padding='same')(enc2)
    enc2 = Conv2D(128, (3,3), activation='relu', padding='same')(enc2)
    
    enc3 = MaxPooling2D(pool_size=(2,2))(enc2)
    enc3 = Conv2D(256, (3,3), activation='relu', padding='same')(enc3)
    enc3 = Conv2D(256, (3,3), activation='relu', padding='same')(enc3)
    
    enc4 = MaxPooling2D(pool_size=(2,2))(enc3)
    enc4 = Conv2D(512, (3,3), activation='relu', padding='same')(enc4)
    enc4 = Conv2D(512, (3,3), activation='relu', padding='same')(enc4)
    
    encoder = MaxPooling2D(pool_size=(2,2))(enc4)
    encoder = Conv2D(1024, (3,3), activation='relu', padding='same')(encoder)
    encoder = Conv2D(1024, (3,3), activation='relu', padding='same')(encoder)
    
    # Expansive Path
    dec4 = UpSampling2D((2,2))(encoder)
    dec4 = concatenate([dec4,enc4])
    dec4 = Conv2D(512, (3,3), activation='relu', padding='same')(dec4)
    dec4 = Conv2D(512, (3,3), activation='relu', padding='same')(dec4)
    
    dec3 = UpSampling2D((2,2))(dec4)
    dec3 = concatenate([dec3,enc3])
    dec3 = Conv2D(256, (3,3), activation='relu', padding='same')(dec3)
    dec3 = Conv2D(256, (3,3), activation='relu', padding='same')(dec3)
    
    dec2 = UpSampling2D((2,2))(dec3)
    dec2 = concatenate([dec2,enc2])
    dec2 = Conv2D(128, (3,3), activation='relu', padding='same')(dec2)
    dec2 = Conv2D(128, (3,3), activation='relu', padding='same')(dec2)
    
    dec1 = UpSampling2D((2,2))(dec2)
    dec1 = concatenate([dec1,enc1])
    dec1 = Conv2D(64, (3,3), activation='relu', padding='same')(dec1)
    
    # c_exp = final feature maps from expansive path
    c_exp = Conv2D(64, (3,3), activation='relu', padding='same')(dec1)
    
    # output layer must change
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c_exp)
    print("outputs shape: ", outputs.shape)
    
    model = Model(inputs=[input_img], outputs=[outputs])
    return model